In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cupy, cudf, cuml

In [ ]:
data_train= pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
data_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
data_train.head()

In [ ]:
data_test.head()

In [ ]:
sample = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
data_test['SalePrice'] = sample['SalePrice']
data_test

In [ ]:
X = data_train.iloc[:,:-1]
columns = X.columns
Y = data_train.iloc[:,-1]
X_test = data_test.iloc[:,:-1]
Y_test = data_test.iloc[:,-1]

In [ ]:
X = X.append(X_test,ignore_index=True)
Y = Y.append(Y_test,ignore_index=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
temp = X.select_dtypes(include = "object").columns
temp = list(temp)
X[temp].apply(lambda col: col.astype(str))

In [ ]:
X_temp = X[temp].apply(lambda col: LabelEncoder().fit_transform(col.astype(str)), axis=0, result_type='expand')

In [ ]:
X = X.drop(temp,axis=1)
X = pd.concat([X,X_temp],axis=1)

In [ ]:
X = X.replace(np.nan,-1)

In [ ]:
X

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rnd_clf = RandomForestRegressor(n_estimators=500)
rnd_clf.fit(X,Y)
importance_dict = {}
for j in X.columns:
    importance_dict[j]=0
for name,score in zip(columns,rnd_clf.feature_importances_):
    importance_dict[name]=importance_dict[name]+score
    print(name,score)
for k in X.columns:
    if(importance_dict[k]<0.0008):
        X=X.drop([k],1)

In [ ]:
sorted_idx = rnd_clf.feature_importances_.argsort()
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 20))
plt.barh(columns[sorted_idx],rnd_clf.feature_importances_[sorted_idx])

In [ ]:
from sklearn.model_selection import train_test_split
X,X_test,Y,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [ ]:
columns=X.columns
len(columns)

In [ ]:
from cuml.preprocessing import StandardScaler
for i in columns :
    scaler = StandardScaler()
    X[i] = scaler.fit_transform(X[i].values.reshape(-1,1))
    X_test[i] = scaler.transform(X_test[i].values.reshape(-1,1))

In [ ]:
X = X.iloc[:,1:]
X_test = X_test.iloc[:,1:]

In [ ]:
caler = StandardScaler()
Y = scaler.fit_transform(Y.values.reshape(-1,1))
Y_test = scaler.transform(Y_test.values.reshape(-1,1))

In [ ]:
from cuml import LinearRegression
from cuml.linear_model import LinearRegression
X_new = cd.DataFrame(X)
Y_new = cd.DataFrame(Y)
lr = LinearRegression(fit_intercept = False,algorithm = "svd")
reg = lr.fit(X_new,Y_new)
preds = lr.predict(X_test)
from cuml.metrics import mean_squared_error
from cuml.metrics import mean_absolute_error
from cuml.metrics import r2_score
MSE = mean_squared_error(Y_test,preds)
MAE = mean_absolute_error(Y_test,preds)
R2 = r2_score(Y_test,preds)
print(MSE,MAE,R2)

In [ ]:
from cuml.linear_model import LogisticRegression
reg = LogisticRegression()
reg.fit(X,Y)
preds = reg.predict(X_test)
from cuml.metrics import mean_squared_error
from cuml.metrics import mean_absolute_error
from cuml.metrics import r2_score
MSE = mean_squared_error(Y_test,preds)
MAE = mean_absolute_error(Y_test,preds)
R2 = r2_score(Y_test,preds)
print(MSE,MAE,R2)

In [ ]:
from cuml import Ridge
from cuml.linear_model import Ridge
ridge = Ridge(alpha=0.5,fit_intercept = True, normalize = False,solver = "svd")
result_ridge = ridge.fit(X, Y)
preds = result_ridge.predict(X_test)
from cuml.metrics import mean_squared_error
from cuml.metrics import mean_absolute_error
from cuml.metrics import r2_score
MSE = mean_squared_error(Y_test,preds)
MAE = mean_absolute_error(Y_test,preds)
R2 = r2_score(Y_test,preds)
print(MSE,MAE,R2)

In [ ]:
from cuml.linear_model import Lasso
ls = Lasso(alpha = 0.05)
result_lasso = ls.fit(X, Y)
preds = result_lasso.predict(X_test)
from cuml.metrics import mean_squared_error
from cuml.metrics import mean_absolute_error
from cuml.metrics import r2_score
MSE = mean_squared_error(Y_test,preds)
MAE = mean_absolute_error(Y_test,preds)
R2 = r2_score(Y_test,preds)
print(MSE,MAE,R2)

In [ ]:
from cuml.linear_model import ElasticNet
enet = ElasticNet(alpha = 0.3, l1_ratio=0.1)
result_enet = enet.fit(X, Y)
preds = result_enet.predict(X_test)
from cuml.metrics import mean_squared_error
from cuml.metrics import mean_absolute_error
from cuml.metrics import r2_score
MSE = mean_squared_error(Y_test,preds)
MAE = mean_absolute_error(Y_test,preds)
R2 = r2_score(Y_test,preds)
print(MSE,MAE,R2)